# Quantum mechanics in one dimension

## TL;DR
Change the sections marked by a lot of `#####` to change the major settings
 - space: grid interval, accuracy and boundary conditions
 - potential
 - preparation of an initial wave function
 - time: evolution time and accuracy
This example script then runs and plots the following for you:
 - show the potential
 - show the eigenstates of the stationary Schrödinger equation
 - show the time evolution of the prepared state

## remarks
- we are using hartree atomic units `hbar = m = e = 1/4pi epsilon_0`
- most of the scripts lines are plotting commands (is on todo list)
- default initial wavefunction is a gaussian wave package moving towards +infinity

## features 
- custom girds
- custom potential
- custom operators
- eigen solver
- time evolution solver

In [ ]:
#import packages 
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
# below is the qm1 package
from qm1.grid import *
from qm1.operators import *
# from qm1.operator_td import *
from qm1.qmsystem import *
from qm1.eigensystem import *
from qm1.wavefunction import *

In [ ]:
# set up a (periodic) grid
bc = "periodic"
grid = UniformGrid(boundary_condition=bc, xmin=-20., xmax=20., num=100)
# define mass
mass = 1.
# define potential to use
potential = BarrierPot(xstart=-1., xstop=+2., vstep=-.1)
# set the potential
qsys = QMSystem(potential, grid, mass)

In [ ]:
import sys
%load_ext autoreload
%autoreload 2

# construct some operators, that act on the wave function 
op_identity = IdentityOp(qsys.grid)
op_position = PositionOp(qsys.grid)
op_momentum = MomentumOp(qsys.grid)
op_kinetic  = KineticOp(qsys)
op_potential_stat = PotentialOp(qsys)

# make the operators more efficient
make_efficient([op_identity, op_kinetic, op_position, op_momentum, op_potential_stat])

# time dependent potential operator (local) - perturbation
op_perturbe = OperatorTD(qsys.grid, func=(lambda x,t: 0.01*x*np.sin(t)))
print('1 op_perturbe', op_perturbe.eval(t=1.).matrix())
print('1 op_potential_stat', op_potential_stat.matrix())

# potential and hamilton operator
op_potential = OperatorTD(qsys.grid)
op_potential = op_perturbe + op_potential_stat
print('2 op_perturbe', op_perturbe.eval(t=1.).matrix())
print('2 op_potential_stat', op_potential_stat.matrix())

print(op_perturbe.eval(t=1.).matrix())
op_hamilton = op_potential + op_kinetic

# show a few operators
op_potential_stat.show(file='op_potential_stat.png')
op_kinetic.show(file='op_kinetic.png')
op_perturbe.show(tgrid=np.linspace(0, 2*np.pi, 20), file='op_perturbe.gif')
op_perturbe.eval(t=1.).show(file='op_perturbe0.png')
op_hamilton.show(tgrid=np.linspace(0, 2*np.pi, 20), file='op_hamilton.gif')

# return the rhs of the schrödinger equation, when lhs is only the time derivative
#  rhs = i hbar H 
op_rhs = OperatorTD(qsys.grid)
op_rhs = op_hamilton * (1j)
# print(op_hamilton.eval(1))
# print(op_rhs.eval(1))
# 


In [ ]:
#######################################################################################
###### get the eigenstates  ###########################################################
#######################################################################################
num_states = 10     # number of states to calculate
#######################################################################################
#######################################################################################
from qm1.eigensystem import Eigensystem

op_hamilton_gs = op_hamilton.eval(t=0.)
print(op_hamilton_gs)

# get eigensystem 
eigsys = Eigensystem(num=num_states, operator=op_hamilton_gs)

# get observables
observables = []
for _i in range(num_states):
  norm              = eigsys.eigstates[_i].expectation_value(op_identity) #"{:.5f}".format()
  position          = eigsys.eigstates[_i].expectation_value(op_position) #"{:.5f}".format()
  position_var      = eigsys.eigstates[_i].variance(op_position) #"{:.5f}".format()
  momentum          = eigsys.eigstates[_i].expectation_value(op_momentum) #"{:.5f}".format()
  energy            = eigsys.eigstates[_i].expectation_value(op_hamilton_gs)  # "{:.5f}".format()
  energy_variance   = eigsys.eigstates[_i].variance(op_hamilton_gs)  # "{:.5f}".format()
  observables.append([norm, position, position_var, momentum, energy, energy_variance])
observables = np.array(observables)
labels = ['state', 'norm', 'position', 'position_var', 'momentum', 'energy', 'energy variance']

# plot
fig, ax = plt.subplots()
colors = [plt.cm.tab10(i) for i in range(num_states)]
ax.grid()
plt.title('eigen states of the hamiltonian')
ax.set_xlabel('position')
ax.set_ylabel('wave function')
for i in reversed(range(num_states)):
  ax.plot(grid.points, eigsys.eigstates[i].func, label='state '+str(i), alpha=((num_states-i)/num_states)**2., color=colors[i])
plt.legend(bbox_to_anchor=(.85,1.),loc=2)
# print table
print('summary of hamiltonians eigenstates')
print([_s.ljust(25) for _s in labels])
for _i,_o in enumerate(observables):
  print(str(_i).ljust(25), ' '.join([str(_e).ljust(25) for _e in _o]))



In [ ]:
#######################################################################################
###### define the initial wavefunction here ###########################################
#######################################################################################
# initial wavepackage
# - must ly in the region/interval [x_min, x_max]
# - will get normalized automatically
# - example = gaussian wave package
kx = +5.           # wave number
sigma = .6        # width of initial wave packet
x0 = -1.           # center of initial wave packet
# initial wave function
def prep_wf(x): return np.exp(-(x-x0)**2 / (2.0 * sigma**2)) * np.exp(-1j * kx * x)
########################################################################################
########################################################################################

In [ ]:
# define the initial WF
prepared_psi = Wavefunction(grid)
prepared_psi.set_via_func(prep_wf)
prepared_psi.show('init_wavefunc.png')

# check the initial expectation values of some observables
print("initial wave func: norm:              ", np.real(prepared_psi.expectation_value(op_identity)))
print("initial wave func: norm-variance:     ", np.real(prepared_psi.variance(op_identity)))
print("initial wave func: energy:            ", np.real(prepared_psi.expectation_value(op_hamilton_gs)))
print("initial wave func: energy-variance:   ", np.real(prepared_psi.variance(op_hamilton_gs)))
print("initial wave func: momentum           ", np.real(prepared_psi.expectation_value(op_momentum)))
print("initial wave func: momentum-variance  ", np.real(prepared_psi.variance(op_momentum)))
print("initial wave func: position           ", np.real(prepared_psi.expectation_value(op_position)))
print("initial wave func: position-variance  ", np.real(prepared_psi.variance(op_position)))

In [ ]:
#######################################################################################
###### define the time span and accuracy ##############################################
#######################################################################################
# time interval and accuracy

#######################################################################################
#######################################################################################b

In [ ]:
# do the actual time evolution
# - depending on the accuracy of time and space - this could take a while

dt = 0.25        # time step
tmax = 10.       # total time
psis, tgrid = prepared_psi.evolve(tmax, dt, op_rhs)


In [ ]:

def print_evolution(psis, tgrid, file):
  import matplotlib.pyplot as plt
  from matplotlib.animation import FuncAnimation
  fig, ax = plt.subplots(figsize=(10, 10))
  ymin = min([np.min(np.abs(_psi.func)**2) for _psi in psis])
  ymax = max([np.max(np.abs(_psi.func)**2) for _psi in psis])

  def animate(i):
    ax.clear()
    ax.set_title('evolution of wavefunction')
    ax.set_xlabel('position')
    ax.set_ylabel('density of the wave function')
    ax.set_ylim((ymin, ymax))
    line = ax.plot(psis[0].grid.points, np.abs(psis[i].func)**2)
    text = ax.text(0.8, 0.9, 'time='+str(tgrid[i]), horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
    return [line, text]
  ani = FuncAnimation(fig=fig, func=animate, frames=len(psis), interval=1000./24.)
  ani.save(file, writer='imagemagick', fps=24)
  plt.close()


print_evolution(psis, tgrid, file='wavefunc.gif')

In [ ]:
# plot the time evolution of the wave function

# todo : reserved for time dependent potentials
# get the time evolution of the coefficients 
coeffs = []
for _i in range(eigsys.num):
  coeff, rest = eigsys.decompose(psis[_i])
  coeffs.append(coeff)
coeffs = np.array(coeffs)


# first plot: the wavefunction in real-space-time
fig, (ax1, ax2) = plt.subplots(2,1)
title =fig.suptitle('')
ax1.set_xlim([x_min, x_max])

# get minimum and maximum value of the wf
max_y = 0.
for _wf in eigsys.eigstates:
  max_y = max(max_y, np.max(np.abs(_wf.func)))
ax1.set_ylim([0, np.sqrt(max_y)])
ax1.set_title('wavefunction')
line1,  = ax1.plot([], [], "k--")  # this line represents the potential and is initially empty
line2,  = ax1.plot([], [])         # this line represents the wave function and is initially empty

# second plot: the coefficients in the eigen"basis"
ax2.set_title('coefficients in eigen system')
boxes = ax2.bar([_i for _i in range(eigsys.num)], [.1 for _ in range(eigsys.num)])         # this line represents the wave function and is initially empty

# todo : reserved for time dependent potentials
# todo : third plot: energy eigenvalues and expectation value

# def the initial state of the plot
def init():
    line1.set_data(grid.points, qsys.pot)                # initial potential (... it stays constant anyway)
    line2.set_data(grid.points, np.abs(prepared_psi.func)**2)    # initial wave function amplitude (... the one you defined above)
    for _i, _b in enumerate(boxes):
      _b.set_height(coeffs[0,_i])
    return [_b for _b in boxes]+[line1, line2]


# def how the plot gets updated when going from plot t to plot t+1
def animate(t):
    line2.set_data(grid.points, np.abs(psis[t].func)**2)       # only update the wavefunction 
    title.set_text('time = {0:2.3f}'.format(t*dt))  # set a new title showing the time
    for _i, _b in enumerate(boxes):
      _b.set_height(coeffs[0, _i])
    return [_b for _b in boxes]+[line1, line2]


# animate
anim = animation.FuncAnimation(fig=fig, func=animate, init_func=init, frames=int(tmax/dt), interval=50, blit=True)


In [ ]:
# Display the animation in the jupyter notebook
HTML(anim.to_jshtml())

In [ ]:
# save the movie to 'mp4' format
mytitle = 'qm1-dynamics'
quality_dpi = 60
if True:
  anim.save(mytitle+'.mp4', fps=15, extra_args=['-vcodec', 'libx264'], dpi=quality_dpi)


In [ ]:
from copy import copy
class InnerData:
  def __init__(self, val=0):
    self.val = val

class OuterData:
  def __init__(self, val=0):
    self.dat = InnerData(val)

  def __add__(self, other):
    # res = copy(self)
    res = OuterData()
    res.dat.val = self.dat.val + other.dat.val
    return res

  def _set(self, val):
    self.dat.val = val

obj1 = OuterData(3)
obj2 = OuterData(2)
obj3 = obj1 + obj2 
print('obj1 val:', obj1.dat.val)
print('obj2 val:', obj2.dat.val)
print('obj3 val:', obj3.dat.val)
obj2._set(5)
print('obj1 val:', obj1.dat.val)
print('obj2 val:', obj2.dat.val)
print('obj3 val:', obj3.dat.val)
